# "World-wide excess deaths in the time of COVID-19"
> "This notebook visualises excess deaths around the world. It's very much work in progress. In fact, it may never progress past its current state. Test"

- toc: true
- branch: master
- badges: true
- comments: false
- author: Bas van Schaik (https://github.com/sj)
- categories: [fastpages, jupyter, covid19]

In [1]:
import pandas as pd
import altair as alt
import numpy as np
import scipy.stats as st

In [69]:
df = pd.read_csv("https://raw.githubusercontent.com/sj/covid19-excess-deaths-data/master/weekly-deaths.csv", sep=';', parse_dates=['first_day', 'last_day'])

## Create separate column containing the year of the measurement. Will be used to split up
## years in different data series in the visualisation.
df['first_day:year'] = df['first_day'].dt.year
df['last_day:year'] = df['last_day'].dt.year

## Create column for day-of-the-year to plot the different series for the different years
df['first_day:yday'] = df['first_day'].dt.strftime('%j').astype(int)
df['last_day:yday'] = df['last_day'].dt.strftime('%j').astype(int)

## Create column for the middle day 
df['middle_day:yday'] = (df['first_day:yday'] + (df['last_day:yday'] - df['first_day:yday']) / 2.0).astype(int)

## To investigate/display the entire dataset, uncomment the following:
#with pd.option_context('display.max_rows', 2000):
display(df)

,country,first_day,last_day,filter,deaths,source,notes,first_day:year,last_day:year,first_day:yday,last_day:yday,middle_day:yday
0,NL,1995-01-02,1995-01-08,NaN,2719,https://opendata.cbs.nl/statline/#/CBS/nl/data...,Previous week straddled two years and was skip...,1995,1995,2,8,5
1,NL,1995-01-09,1995-01-15,NaN,2823,NaN,NaN,1995,1995,9,15,12
2,NL,1995-01-16,1995-01-22,NaN,2609,NaN,NaN,1995,1995,16,22,19
3,NL,1995-01-23,1995-01-29,NaN,2664,NaN,NaN,1995,1995,23,29,26
4,NL,1995-01-30,1995-02-05,NaN,2577,NaN,NaN,1995,1995,30,36,33
5,NL,1995-02-06,1995-02-12,NaN,2536,NaN,NaN,1995,1995,37,43,40
6,NL,1995-02-13,1995-02-19,NaN,2551,NaN,NaN,1995,1995,44,50,47
7,NL,1995-02-20,1995-02-26,NaN,2510,NaN,NaN,1995,1995,51,57,54
8,NL,1995-02-27,1995-03-05,NaN,2490,NaN,NaN,1995,1995,58,64,61
9,NL,1995-03-06,1995-03-12,NaN,2770,NaN,NaN,1995,1995,65,71,68


In [93]:
# Select pre-COVID data (everything before 2020) and build confidence range
df_precovid = df[df['first_day:year'] < 2020]

## There must be a neater way to do this with Pandas, but for now we'll use this way to calculate 
## a rolling-window mean and stddev for every day of the year.
window_prefix_postfix = 14 # 5 days before every day, the day itself, 5 days after every day

ydays_means_stdevs = []
means = []
stdevs = []
for yday in range (1, 366):
    
    
    window_first = yday - window_prefix_postfix - 1
    window_last = yday + window_prefix_postfix
    
    window_ydays = range(window_first + 365, window_last + 365)
    window_ydays = [(yday % 365) + 1 for yday in window_ydays]
    
    # If window straddles multiple years, also include data from 366th day (for leap years)
    if window_ydays[0] > window_ydays[-1]:
        window_ydays.append(366)
        
    #print(str(yday) + ": " + str(window_ydays))
    
    df_relevant = df_precovid[df_precovid['middle_day:yday'].isin(window_ydays)]
    
    ydays_means_stdevs.append([yday, df_relevant['deaths'].mean(), df_relevant['deaths'].std(), df_relevant['deaths'].count()])

# Account for leap years. Mean and stddev of day 366 is same as for day 365
ydays_means_stdevs.append(ydays_means_stdevs[-1])
ydays_means_stdevs[-1][0] = 366
df_ydays_means_stdevs = pd.DataFrame(ydays_means_stdevs, columns = ["yday", "mean", "stdev", "sample_size"])

# Define 68% (1 stdev) and 95% (2 stdev) confidence intervals
df_ydays_means_stdevs['ci_68_lower'] = df_ydays_means_stdevs['mean'] - df_ydays_means_stdevs['stdev']
df_ydays_means_stdevs['ci_68_upper'] = df_ydays_means_stdevs['mean'] + df_ydays_means_stdevs['stdev']
df_ydays_means_stdevs['ci_95_lower'] = df_ydays_means_stdevs['mean'] - 2 * df_ydays_means_stdevs['stdev']
df_ydays_means_stdevs['ci_95_upper'] = df_ydays_means_stdevs['mean'] + 2 * df_ydays_means_stdevs['stdev']

df_ydays_means_stdevs
# Combine yday, mean, stdev into new dataframe

#print(means)
#print(stdevs)
#df_precovid.rolling()
#st.t.interval(0.95, len(df_precovid)-1, loc=np.mean(df_precovid['deaths']))

#alt.Chart(df_precovid).mark_line().encode(
#    alt.X('first_day:yday', type='quantitative'),
#    alt.Y('deaths', type='quantitative')
#)

,yday,mean,stdev,sample_size,ci_68_lower,ci_68_upper,ci_95_lower,ci_95_upper
0,1,3000.297619,255.638241,84,2744.659378,3255.935860,2489.021137,3511.574101
1,2,3000.190476,256.394438,84,2743.796038,3256.584914,2487.401600,3512.979352
2,3,3003.333333,253.830763,84,2749.502570,3257.164096,2495.671807,3510.994859
3,4,3006.904762,257.439880,84,2749.464881,3264.344642,2492.025001,3521.784523
4,5,3003.273810,264.518902,84,2738.754908,3267.792711,2474.236006,3532.311613
5,6,3008.416667,266.642679,84,2741.773988,3275.059346,2475.131309,3541.702025
6,7,3007.821429,266.650268,84,2741.171161,3274.471696,2474.520893,3541.121964
7,8,3017.571429,269.975380,84,2747.596049,3287.546808,2477.620669,3557.522188
8,9,3018.761905,267.963926,84,2750.797979,3286.725831,2482.834053,3554.689756
9,10,3023.761905,267.219806,84,2756.542099,3290.981710,2489.322293,3558.201516


In [118]:
legend_selection = alt.selection_multi(fields=['first_day:year'], bind='legend')

# Plots a single line for every year in an interactive chart
df['line_size'] = df.apply(lambda row: 0.5 if row['first_day:year'] != 2020 else 1, axis = 1)

df_recent = df[df['first_day:year'] > 2015]
line = alt.Chart(df).mark_line().encode(
    alt.X(
        'first_day:yday', type='quantitative', 
        title='Day of the year (week following)',
        scale=alt.Scale(domain=(1, 366))
    ),
    alt.Y('deaths', type='quantitative', title='Number of deaths'),
    
    # Documentation on available colour schemes in Vega: https://vega.github.io/vega/docs/schemes/
    alt.Color('first_day:year', type='ordinal', scale=alt.Scale(scheme='yellowgreenblue'), title='Year'),
    
    # Enable selection of series (years) from the legend
    opacity=alt.condition(legend_selection, alt.value(1), alt.value(0.3)),
    
    # Display tooltips
    tooltip=[alt.Tooltip('country', title='Country'), 
             alt.Tooltip('first_day', title='Week starting'),
             alt.Tooltip('deaths', title='Number of deaths')
    ],
    
    size = alt.condition(legend_selection, alt.value(1.5), alt.value(0.7))
).properties(
    width=800
).interactive().add_selection(legend_selection) #.add_selection(alt.selection_interval(bind='scales'))

ci68_band = alt.Chart(df_ydays_means_stdevs).mark_area(
    opacity = 0.3,
    color = 'goldenrod'
).encode(
    x = 'yday',
    y = 'ci_68_lower',
    y2 = 'ci_68_upper',

)

ci95_band = alt.Chart(df_ydays_means_stdevs).mark_area(
    opacity = 0.2,
    color = 'goldenrod'
).encode(
    x = 'yday',
    y = 'ci_95_lower',
    y2 = 'ci_95_upper',

)

mean_line = alt.Chart(df_ydays_means_stdevs).mark_line(
    strokeDash=[3,5],
    color = 'black',
    opacity = 0.5,
    size = 1
).encode(
    x = 'yday',
    y = 'mean'
)

#band = alt.Chart(df).mark_errorband(extent='ci').encode(
#    x = 'first_day',
#    y = 'deaths'
#)

line + ci68_band + ci95_band + mean_line

alt.LayerChart(...)